In [1]:
!cp /content/drive/MyDrive/kvasir_resnet.h5 .

In [2]:
import cv2
import numpy as np
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt
import os

from tensorflow.keras.preprocessing.image import img_to_array

import tensorflow.keras as K
     

from tensorflow.keras.models import Model
     

In [3]:
model = keras.models.load_model("kvasir_resnet.h5")
last_conv_layer = next(x for x in model.layers[::-1] if isinstance(x, K.layers.Conv2D))

In [4]:
last_conv_layer.name

'conv5_block3_3_conv'

In [6]:
# Load the model
model = keras.models.load_model("kvasir_resnet.h5")

# Define the gradcam function
def gradcam(model, img, layer_name):
  # Define the gradient model
  grad_model = keras.models.Model([model.inputs], [model.get_layer(layer_name).output, model.output])

  # Compute the gradient of the top predicted class for our input image
  with tf.GradientTape() as tape:
    conv_output, predictions = grad_model(np.array([img]))
    loss = predictions[:, np.argmax(predictions[0])]
    grads = tape.gradient(loss, conv_output)[0]

  # Compute the guided gradients
  guided_grads = (tf.cast(conv_output > 0, "float32") * tf.cast(grads > 0, "float32") * grads)

  # Get the convolution output and guided gradients
  conv_output = conv_output[0]
  guided_grads = guided_grads[0]

  # Compute the weights using global average pooling
  weights = tf.reduce_mean(guided_grads, axis=(0, 1))

  # Create a heatmap using the weights and conv output
  cam = np.zeros(dtype=np.float32, shape=conv_output.shape[0:2])
  for i, w in enumerate(weights):
    cam += w * conv_output[:, :, i]

  # Normalize the heatmap
  cam = cv2.resize(cam.numpy(), (img.shape[1], img.shape[0]))
  cam = np.maximum(cam, 0)
  heatmap = cam / np.max(cam)

  # Convert the heatmap to RGB
  heatmap = cv2.applyColorMap(np.uint8(255 * heatmap), cv2.COLORMAP_JET)

  # Overlay the heatmap on the original image
  overlay = cv2.addWeighted(cv2.cvtColor(img.astype('uint8'), cv2.COLOR_RGB2BGR), 0.7, heatmap, 0.3, 0)

  return overlay


# Define the video input and output paths
video_input_path = "video.mp4"
video_output_path = "output_video.mp4"

# Define the layer name to visualize
layer_name = "conv5_block3_out"

# Load the video
cap = cv2.VideoCapture(video_input_path)

# Get the video properties
fps = cap.get(cv2.CAP_PROP_FPS)
width = 224
height = 224

# Define the codec and create VideoWriter object
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(video_output_path, fourcc, fps, (224, 224))

# Read until video is completed
while(cap.isOpened()):
  # Read a frame from the video
  ret, frame = cap.read()
  
  if ret == True:
    # Preprocess the frame
    #frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    #frame = cv2.resize(frame, (128, 128))
    #frame = np.array(frame, dtype=np.float32)


    frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    frame = cv2.resize(frame, (224, 224))
    frame = np.array(frame, dtype=np.float32)
    #frame = frame / 255.0  # normalize pixel values to [0, 1]
    # Apply any other preprocessing steps that you used during training of the model
    #frame = keras.applications.resnet50.preprocess_input(frame)
       # Apply the gradcam function to the frame
    guided_cam = gradcam(model, frame, layer_name)
    
    # Convert the guided_cam to BGR
    guided_cam = cv2.cvtColor(guided_cam, cv2.COLOR_RGB2BGR)
    
    # Write the frame to the output video
    out.write(guided_cam)
    
    # Display the resulting frame
    #cv2.imshow('Frame',guided_cam)
    
    # Press Q on keyboard to exit
    #if cv2.waitKey(25) & 0xFF == ord('q'):
      #break
  
  # Break the loop
  else:
    break

# Release the video capture and writer objects
cap.release()
out.release()

# Close all the frames
cv2.destroyAllWindows()
    
     